In [92]:
num_sample = 500
num_burn = 200
sample_size = 800
n_cpu = 100

min_degree = 1
max_degree = 4

In [93]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1*min_degree,-2*min_degree,2,2,0.1,-1,0.1,2,0.1,0])  # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj1, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::5]
A_chain = A_chain[::5]
L_chain = L_chain[::5]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

1 4


(-0.021176363765848714,
 0.41631250000000003,
 0.6272375,
 0.34178288653114913,
 0.19798254982088362)

In [94]:
res_truth1 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0
)
res_truth2 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.3,
    R=100,
    burnin_R=100,
    seed=0
)

ground_truth1 = {}
ground_truth1['average'] = res_truth1['average']
ground_truth1['direct'] = res_truth1['direct_effect']
ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
ground_truth1['spillover_effect'] = res_truth1['spillover_effect']
ground_truth1['psi_1_gamma'] = res_truth1['psi_1_gamma']
ground_truth1['psi_0_gamma'] = res_truth1['psi_0_gamma']
ground_truth1['psi_zero'] = res_truth1['psi_zero']

ground_truth1

{'average': 0.6514624999999999,
 'direct': -0.22871249999999987,
 'indirect': 0.1429499999999998,
 'spillover_effect': 0.3007124999999998,
 'psi_1_gamma': 0.5821875,
 'psi_0_gamma': 0.8108999999999998,
 'psi_zero': 0.5101875}

In [95]:
from utils import *
from run_pll import run_dr_em_raw

treat_p = 0.7
mispec = 'outcome'
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'mispec': mispec} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
results1 = res_list_array_dr.copy()

delete_all_files_in_folder(f'run/run_dr_em_raw')

from utils import run_pll
from run_pll import run_dr_em_raw

treat_p = 0.3
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'mispec': mispec} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
results2 = res_list_array_dr.copy()

delete_all_files_in_folder(f'run/run_dr_em_raw')

compute_stats(results1, results2, ground_truth1, adj1)

Multiprocessing <function run_dr_em_raw at 0x7f4c8f9cc720> in 100 tasks, with 100 processes...
Multiprocessing finished.
Multiprocessing <function run_dr_em_raw at 0x7f4c8f9cc720> in 100 tasks, with 100 processes...
Multiprocessing finished.


100%|██████████| 100/100 [00:06<00:00, 16.44it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.92, 0.95, 0.92, 0.91, 0.92, 0.94, 0.91]),
 'bias': array([-0.00717862, -0.02206337,  0.00918291,  0.02385599, -0.01313638,
         0.00892699, -0.01492901]),
 'mse': array([0.00331698, 0.00873615, 0.0024492 , 0.00864149, 0.006847  ,
        0.00116539, 0.0080453 ]),
 'var': array([0.00326545, 0.00824935, 0.00236487, 0.00807238, 0.00667443,
        0.0010857 , 0.00782242]),
 'ci_length': array([0.20851508, 0.3221672 , 0.2055737 , 0.34014656, 0.28891793,
        0.12492953, 0.35424751]),
 'true_effect': array([ 0.6514625, -0.2287125,  0.14295  ,  0.3007125,  0.5821875,
         0.8109   ,  0.5101875])}

## Compare with AUTOG

In [96]:
from run_pll import run_autognet_raw

mispec = 'outcome'

if mispec == 'outcome':
    np.random.seed(0)
    L_chain_noise = np.random.binomial(n=1, p=0.2, size=L_chain.shape)

    treat_p = 0.7
    args = [{'Y_chain': Y_chain,
            'A_chain': A_chain,
            'L_chain': L_chain_noise,
            'adj': adj1,
            'i': i,
            'treatment_allocation': treat_p} for i in range(len(Y_chain))]

    print(Y_chain[10,:20])

    res_list_dr = run_pll(run_autognet_raw, args, processes=n_cpu)
    res_list_array_dr = np.array(res_list_dr)
    results1 = res_list_array_dr.copy()
    
    delete_all_files_in_folder(f'run/run_autog_raw')


    treat_p = 0.3
    args = [{'Y_chain': Y_chain,
            'A_chain': A_chain,
            'L_chain': L_chain,
            'adj': adj1,
            'i': i,
            'treatment_allocation': treat_p} for i in range(len(Y_chain))]

    print(Y_chain[10,:20])

    res_list_dr = run_pll(run_autognet_raw, args, processes=n_cpu)
    res_list_array_dr = np.array(res_list_dr)
    results2 = res_list_array_dr.copy()
    
    delete_all_files_in_folder(f'run/run_autog_raw')


compute_stats_ag(results1, results2, ground_truth1)

[1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1]
Multiprocessing <function run_autognet_raw at 0x7f4c8f9cc7c0> in 100 tasks, with 100 processes...
Multiprocessing finished.
[1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1]
Multiprocessing <function run_autognet_raw at 0x7f4c8f9cc7c0> in 100 tasks, with 100 processes...
Multiprocessing finished.


{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.0786545 ,  0.1822705 ,  0.18683575,  0.09720175,  0.1335365 ,
        -0.048734  , -0.14593575]),
 'mse': array([0.00650164, 0.03396602, 0.03652899, 0.01057727, 0.01841993,
        0.00260644, 0.02254668]),
 'var': array([0.00031511, 0.00074348, 0.00162139, 0.00112909, 0.00058793,
        0.00023143, 0.00124944]),
 'ground_truth': array([ 0.6514625, -0.2287125,  0.14295  ,  0.3007125])}

'bias': array([-0.00717862, -0.02206337,  0.00918291,  0.02385599, -0.01313638,
         0.00892699, -0.01492901]),
 'mse': array([0.00331698, 0.00873615, 0.0024492 , 0.00864149, 0.006847  ,
        0.00116539, 0.0080453 ]),

In [97]:
from utils import *
from run_pll import run_dr_em_raw

treat_p = 0.7
mispec = None
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'mispec': mispec} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
results1 = res_list_array_dr.copy()

delete_all_files_in_folder(f'run/run_dr_em_raw')

from utils import run_pll
from run_pll import run_dr_em_raw

treat_p = 0.3
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'mispec': mispec} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
results2 = res_list_array_dr.copy()

delete_all_files_in_folder(f'run/run_dr_em_raw')

compute_stats(results1, results2, ground_truth1, adj1)

Multiprocessing <function run_dr_em_raw at 0x7f4c8f9cc720> in 100 tasks, with 100 processes...
Multiprocessing finished.
Multiprocessing <function run_dr_em_raw at 0x7f4c8f9cc720> in 100 tasks, with 100 processes...
Multiprocessing finished.


100%|██████████| 100/100 [00:06<00:00, 16.42it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.92, 0.93, 0.97, 0.94, 0.9 , 0.91, 0.95]),
 'bias': array([-0.00432278, -0.00823515,  0.00152035,  0.0025803 , -0.00613208,
         0.00210307, -0.00047722]),
 'mse': array([0.00258915, 0.00682003, 0.00173371, 0.00386034, 0.00527241,
        0.00108464, 0.00347572]),
 'var': array([0.00257046, 0.00675222, 0.0017314 , 0.00385368, 0.0052348 ,
        0.00108022, 0.00347549]),
 'ci_length': array([0.16701138, 0.25484955, 0.14867103, 0.22670438, 0.22816921,
        0.10317431, 0.22941625]),
 'true_effect': array([ 0.6514625, -0.2287125,  0.14295  ,  0.3007125,  0.5821875,
         0.8109   ,  0.5101875])}

'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.96, 1.  , 0.92, 0.92, 0.97, 0.85, 0.88]),
 'bias': array([ 2.31398247e-05,  7.83431262e-03, -2.28271532e-03, -1.75841679e-02,
         3.59336098e-05, -7.79837901e-03,  9.78578890e-03]),
 'mse': array([0.00176229, 0.00346415, 0.00497262, 0.03678627, 0.00315528,
        0.00095473, 0.03662267]),
 'var': array([0.00176229, 0.00340277, 0.00496741, 0.03647707, 0.00315528,
        0.00089392, 0.03652691]),
 'ci_length': array([0.16035535, 0.25128148, 0.20171466, 0.55201123, 0.22136661,
        0.10689214, 0.54127235]),
 'true_effect': array([ 0.36545  ,  0.41975  , -0.1044375, -0.231575 ,  0.4937125,
         0.0739625,  0.3055375])}